<a href="https://colab.research.google.com/github/RushiKawade/FareShare/blob/master/MyCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/RushiKawade/Librosa_feature_extract.git
! ls

Cloning into 'Librosa_feature_extract'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.
Librosa_feature_extract  sample_data


In [65]:
from glob import glob
import numpy as np
import os
import fnmatch
import random
import librosa
import time
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras import backend as K
from keras.initializers import glorot_normal
from collections import Counter

random.seed(a=1)
np.random.seed(2)
# use local path in case of local run of program.
data_dir = './Librosa_feature_extract/'
def get_audio_files(path, extension='wav'):
    files={}
    for root, dirnames, filenames in os.walk(path):
        for dirname in dirnames:
            for root2, dirnames2, filenames2 in os.walk(path+dirname):
                for filename3 in fnmatch.filter(filenames2, '*.'+extension):
                    files[path+dirname+"/"+filename3]=filename3
    return files
ext = 'mp3'
filei = get_audio_files(data_dir,ext)
#print(filei)
#print('Number of data files loaded: '+ str(len(audio_files)))

def make_category(files):
    cats={}
    
    speakers=list(set(files.values()))
    nclasses=len(speakers)
    onehotmat=np.eye(nclasses)
    for i in range(nclasses):
        cats[speakers[i]]=onehotmat[i,:]
    return cats

res_cats = make_category(filei)

print(res_cats)








{'ramesh.mp3': array([1., 0., 0., 0.]), 'suresh.mp3': array([0., 1., 0., 0.]), 'sham.mp3': array([0., 0., 1., 0.]), 'ram.mp3': array([0., 0., 0., 1.])}


In [0]:
def get_features(y, sr):
    	y = y[0:80000] 	# analyze just 80k
    	#S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    	#S = librosa.feature.mfcc(y, sr=sr)
    	#log_S = librosa.logamplitude(S, ref_power=np.max)
    	S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    	log_S = librosa.amplitude_to_db(S)
    	mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    	delta_mfcc = librosa.feature.delta(mfcc)
    	delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    	#mean_mfcc = np.mean(S, 1)
    	#mean_mfcc = (mean_mfcc-np.mean(mean_mfcc))/np.std(mean_mfcc)
    	#var_mfcc = np.var(S, 1)
    	#var_mfcc = (var_mfcc-np.mean(var_mfcc))/np.std(var_mfcc)
    	#feature_vector = np.concatenate((mean_mfcc, var_mfcc))
    	feature_vector = np.concatenate((np.mean(mfcc,1), np.mean(delta_mfcc,1), np.mean(delta2_mfcc,1)))
    	feature_vector = (feature_vector-np.mean(feature_vector)) / np.std(feature_vector)
    	return feature_vector

In [0]:

X=[]
Y=[]
keys=list(filei.keys())
random.shuffle(keys)
for i,key in enumerate(keys):
    aud,s=librosa.load(key)
    mfcc=get_features(aud,s)
    X.append(mfcc)
    speaker=filei[key]
    Y.append(res_cats[speaker])
  

In [82]:
print(X)
print(Y)

[array([-6.02137744,  1.36388054,  0.47387657,  0.37297016,  0.37356463,
        0.19095628,  0.08220292,  0.11917424,  0.17926692,  0.18846693,
        0.17209399,  0.10266771,  0.06560932,  0.09251948,  0.09347036,
        0.09193266,  0.0891787 ,  0.08743868,  0.08826338,  0.08965792,
        0.08881632,  0.08703695,  0.08789908,  0.09066228,  0.09109448,
        0.08872067,  0.09362332,  0.09370567,  0.09078459,  0.08929289,
        0.08921146,  0.08921298,  0.08882543,  0.08832042,  0.08818886,
        0.08886681,  0.09001801,  0.09039881,  0.089507  ]), array([-6.05321706e+00,  1.05546651e+00,  6.61720002e-01,  5.91904916e-01,
        2.25579660e-01,  2.33050767e-01,  6.78494967e-02,  1.38311036e-01,
        1.87414845e-01,  6.93429300e-04,  8.68036859e-02,  6.21923089e-02,
        5.40841701e-02,  1.02933809e-01,  1.05179152e-01,  1.02475279e-01,
        1.02319995e-01,  1.05181677e-01,  1.02215927e-01,  1.04040424e-01,
        1.03119068e-01,  1.02951234e-01,  1.04778553e-01,  

In [86]:
num_classes=4
nb_filter1=16
nb_filter2=8
nb_filter3=4
filter_length=3
filter_length2=2
length=39
model=Sequential()

model.add(Conv1D(filters=nb_filter1, kernel_size=filter_length, activation='relu', input_shape=(length, 1)))
#model.add(MaxPooling1D(pool_size=1, strides=None, padding='valid'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=nb_filter2, kernel_size=filter_length2, activation='relu'))
#model.add(MaxPooling1D(pool_size=1, strides=None, padding='valid'))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu',kernel_initializer=glorot_normal(2), kernel_constraint=None))

model.add(Dropout(0.5)) # CHECK IF NEEDED????

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mae'])

model.summary()




Xl=np.atleast_3d(np.array(X))
Yl=np.array(Y)

nb_samples=Xl.shape[0]
test_size = int((0.02 * nb_samples+1))

X_train, X_test, Y_train, Y_test = Xl[:-test_size], Xl[-test_size:], Yl[:-test_size], Yl[-test_size:]

model.fit(X_train, Y_train, nb_epoch=60, batch_size=20, validation_data=(X_test, Y_test))

pred = model.predict(X_test)
predonehot = np.zeros_like(pred)
predonehot[np.arange(len(pred)), pred.argmax(1)] = 1
predmax=pred.argmax(1)
testmax=Y_test.argmax(1)
cmp=np.stack((predmax.T,testmax.T), axis=1)
#print predonehot
#print Y_test
check=np.abs(cmp[:,0]-cmp[:,1])
cnts=Counter(check)
no_zer=cnts[0]
print ("No of correct classification: " + str(no_zer))
print ("Percentage: " + str(float(no_zer*100)/float(check.shape[0])))


model.save('model60epoch.h5')

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])



Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 37, 16)            64        
_________________________________________________________________
dropout_4 (Dropout)          (None, 37, 16)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 36, 8)             264       
_________________________________________________________________
dropout_5 (Dropout)          (None, 36, 8)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               36992     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)             

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.





Train on 3 samples, validate on 1 samples
Epoch 1/60





3/3 [==============================] - 1s 310ms/step - loss: 1.5503 - mean_absolute_error: 0.3938 - val_loss: 1.2372 - val_mean_absolute_error: 0.3549
Epoch 2/60
3/3 [==============================] - 0s 2ms/step - loss: 1.3802 - mean_absolute_error: 0.3741 - val_loss: 1.2622 - val_mean_absolute_error: 0.3585
Epoch 3/60
3/3 [==============================] - 0s 2ms/step - loss: 1.4423 - mean_absolute_error: 0.3812 - val_loss: 1.2874 - val_mean_absolute_error: 0.3620
Epoch 4/60
3/3 [==============================] - 0s 2ms/step - loss: 1.3379 - mean_absolute_error: 0.3677 - val_loss: 1.3141 - val_mean_absolute_error: 0.3656
Epoch 5/60
3/3 [==============================] - 0s 2ms/step - loss: 1.4821 - mean_absolute_error: 0.3854 - val_loss: 1.3406 - val_mean_absolute_error: 0.3692
Epoch 6/60
3/3 [==============================] - 0s 2ms/step - loss: 1.5948 - mean_absolute_error: 0.3968 - val_loss: 1.3712 - val_mean_absolute_er